In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import gaussian_kde

from skfda.datasets import make_gaussian_process
from skfda.misc.covariances import Exponential, Gaussian, Brownian
from skfda.misc.metrics import l2_distance, l2_norm
from skfda.preprocessing.dim_reduction.feature_extraction import FPCA

Idea: create three covariance functions and sample Gaussian processes. Then do FPCA and compute Functional Graphical Lasso. Let the variables be called $v_1$, $v_2$ and $v_3$. 

In [ ]:
n_var = 3
n_ts = 100
n_samples = 10**3

# define three variables
cov1 = Gaussian(variance=1.0, length_scale=5.0)
cov2 = Gaussian(variance=1.0, length_scale=4.0)
cov3 = Brownian(variance=0.2)

all_cov = [cov1, cov2, cov3]

In [ ]:
# plot covariance functions
_ = cov1.heatmap()
_ = cov2.heatmap()
_ = cov3.heatmap()

In [ ]:
# sample gaussian processes
samples = list()
labels = list()

for j in np.arange(n_var):

    _ds = make_gaussian_process(
            n_samples=n_samples,
            n_features=n_ts,
            start=0.0,
            stop=25.0,
            cov=all_cov[j],
            random_state=20
            )
    
    
    _lb = np.array([f'v{j}'] * n_samples)
    
    samples.append(_ds)
    labels.append(_lb)


In [ ]:
# plot sampled time series

colors = ['darkred', 'C1', 'grey']

fig, axs = plt.subplots(n_var,1, figsize=(10,12))

for j in range(n_var):
    ax = axs.ravel()[j]
    samples[j].plot(axes=ax, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=0.3, lw=0.95)
    ax.set_title(f'Sampled time series for v{j}')

In [ ]:
# do FPCA, reconstruct and plot deviation
q = 5
fig, axs = plt.subplots(n_var, 2, figsize=(20,12))

for j in range(n_var):
    this_ds = samples[j]
    # do FPCA
    fpca = FPCA(n_components=q)
    fpca.fit(this_ds)
    # reconstruct ds from FPCA
    recov_ds = fpca.inverse_transform(fpca.transform(this_ds))
    
    diff = this_ds - recov_ds
    
    ax = axs[j,0]
    recov_ds.plot(axes=ax, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=0.3, lw=0.95)
    ax.set_title(f'FPCA transformed time series for v{j}')
    
    ax2 = axs[j,1]
    diff.plot(axes=ax2, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=0.3, lw=0.95)
    ax2.set_title(f'Recovery error of FPCA for v{j}')
    ax2.set_ylim(ax.get_ylim())
    
    

In [ ]:
# plot error for increasing numper of FPCA components

this_ds = samples[0]
all_err = list()
all_q = range(4,10)

for k in range(len(all_q)):
    
    fpca = FPCA(n_components=all_q[k])
    fpca.fit(this_ds)

    print("Data shape after FPCA transformation: ", fpca.transform(this_ds).shape)
    # reconstruct ds from FPCA
    recov_ds = fpca.inverse_transform(fpca.transform(this_ds))
    this_err = l2_distance(this_ds,recov_ds) / l2_norm(this_ds)

    all_err.append(np.median(this_err))
    
plt.figure()
plt.plot(all_q, all_err, c='darkgray', lw = 4, marker='p', markersize=9, markeredgecolor='k')
plt.xlabel('Number of FPCA components')
plt.ylabel('Median l2 error')
plt.grid(ls = '-', lw = .5) 

## Functional Graphical Lasso

Now, do for each variable a FPCA with $n_{comp}$ components. Then, concatenate the FPCA coefficients for each sample and compute correlations. With this, we are ready to compute Functional Graphical Lasso.

If we choose $n_{comp}=8$, we expect $3\cdot8=24$ variables in total.

In [ ]:
from gglasso.helper.basic_linalg import scale_array_by_diagonal

n_comp = 8
all_traf = list()

# compute FPCA coefficients for each variable
for j in range(n_var):
    this_ds = samples[j]
    fpca = FPCA(n_components=n_comp)
    fpca.fit(this_ds)
    _traf = fpca.transform(this_ds)
    
    # for second variable, permute fpca components in order to get off-diagonal correlation wrt first variable
    if j==1:
        _perm = np.random.permutation(n_comp)
        _traf = _traf[:,_perm]
        
    all_traf.append(_traf)
    
fpca_samples = np.hstack(all_traf)
print("(N,p) = ", fpca_samples.shape)

# compute covariances
S = np.cov(fpca_samples.T)
# scale to correlations
S = scale_array_by_diagonal(S)

print("S has shape ", S.shape)



In [ ]:
# plot samples
fig, ax = plt.subplots()
sns.heatmap(fpca_samples, ax=ax, alpha=1, vmin=-10, vmax=10, cmap='coolwarm',cbar=True)
ax.vlines([(j+1)*n_comp for j in range(n_var)], 0, n_samples, color='k', lw=4)

# plot heatmap
fig, ax = plt.subplots()
sns.heatmap(S, ax=ax, cmap="coolwarm", vmax=.1, vmin=-.1, alpha=1.)
ax.hlines([(j+1)*n_comp for j in range(n_var)], 0, n_var*n_comp, color='k', lw=3)
ax.vlines([(j+1)*n_comp for j in range(n_var)], 0, n_var*n_comp, color='k', lw=3)
